In [1]:
import json
from ultralytics import YOLO

In [ ]:
# Load the JSON file
with open('/Users/awaischoudhry/Documents/Advanced Analytics/Project/CarDD_COCO/annotations/instances_test2017.json', 'r') as f:
    data = json.load(f)

# Extract necessary information
images = {image['id']: image for image in data['images']}
categories = {category['id']: category['name'] for category in data['categories']}
annotations = data['annotations']

# Directory to save YOLO-formatted labels
output_dir = '/Users/awaischoudhry/Documents/Advanced Analytics/Project/CarDD_COCO/labels/test'

# Ensure output directory exists
import os
os.makedirs(output_dir, exist_ok=True)

# Convert annotations to YOLO format
for annotation in annotations:
    image_id = annotation['image_id']
    image = images[image_id]
    image_width, image_height = image['width'], image['height']
    bbox = annotation['bbox']

    # Convert to YOLO format
    x_center = (bbox[0] + bbox[2] / 2) / image_width
    y_center = (bbox[1] + bbox[3] / 2) / image_height
    width = bbox[2] / image_width
    height = bbox[3] / image_height
    class_id = annotation['category_id'] - 1  # YOLO uses 0-indexed class IDs

    # Create label file
    label_file_path = os.path.join(output_dir, f"{image['file_name'].split('.')[0]}.txt")
    with open(label_file_path, 'a') as label_file:
        label_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# Check output directory for YOLO labels
os.listdir(output_dir)

In [2]:
# Set training parameters
#various augmentation hyperparameters are pre-set. No reason for us to diverge as of now.
train_params = {
    "data": "/Users/awaischoudhry/Documents/Advanced Analytics/Project/CarDD_COCO/data.yaml",      # Path to your dataset configuration file
    "epochs": 20,             # Number of training epochs
    "patience": 5,
    "batch": -1,         # Optimized Batch size
    "imgsz": 640,            # Image size (640x640)
    "scale": 0.5,
    "device": "mps",         # Use Apple MPS backend
    "cache": True,
    "amp": True,
    "pretrained": True,
    "optimizer": "auto",
    "workers": 2,            # Number of dataloader workers
    "save_period": 5,       # Save checkpoint every 10 epochs
    "project": "runs/train", # Directory to save results
    "name": "v11n_withback_1",          # Experiment name
    "plots": True,
    "val": True,
    "weight_decay": 0.0005,
    "close_mosaic": 5,
}

In [3]:
model = YOLO("yolo11n.pt") #https://github.com/ultralytics/ultralytics

In [5]:
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}, requires_grad={param.requires_grad}")

model.model.0.conv.weight: torch.Size([16, 3, 3, 3]), requires_grad=True
model.model.0.bn.weight: torch.Size([16]), requires_grad=True
model.model.0.bn.bias: torch.Size([16]), requires_grad=True
model.model.1.conv.weight: torch.Size([32, 16, 3, 3]), requires_grad=True
model.model.1.bn.weight: torch.Size([32]), requires_grad=True
model.model.1.bn.bias: torch.Size([32]), requires_grad=True
model.model.2.cv1.conv.weight: torch.Size([32, 32, 1, 1]), requires_grad=True
model.model.2.cv1.bn.weight: torch.Size([32]), requires_grad=True
model.model.2.cv1.bn.bias: torch.Size([32]), requires_grad=True
model.model.2.cv2.conv.weight: torch.Size([64, 48, 1, 1]), requires_grad=True
model.model.2.cv2.bn.weight: torch.Size([64]), requires_grad=True
model.model.2.cv2.bn.bias: torch.Size([64]), requires_grad=True
model.model.2.m.0.cv1.conv.weight: torch.Size([8, 16, 3, 3]), requires_grad=True
model.model.2.m.0.cv1.bn.weight: torch.Size([8]), requires_grad=True
model.model.2.m.0.cv1.bn.bias: torch.Size([

In [6]:
# Train the model
model.train(**train_params)

New https://pypi.org/project/ultralytics/8.3.57 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.56 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/Users/awaischoudhry/Documents/Advanced Analytics/Project/CarDD_COCO/data.yaml, epochs=20, time=None, patience=5, batch=-1, imgsz=640, save=True, save_period=5, cache=True, device=mps, workers=2, project=runs/train, name=v11n_withback_12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=5, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=

train: Scanning /Users/awaischoudhry/Documents/Advanced Analytics/Project/CarDD_COCO/labels/train.cache... 2816 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2816/2816 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (2.2GB RAM): 100%|██████████| 2816/2816 [00:04<00:00, 581.72it/s]

AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch:  ⚠️ intended for CUDA devices, using default batch-size 16


...

                   all        810       1744      0.629      0.624      0.624      0.496



20 epochs completed in 9.738 hours.
Optimizer stripped from runs/train/v11n_withback_12/weights/last.pt, 5.5MB
Optimizer stripped from runs/train/v11n_withback_12/weights/best.pt, 5.5MB

Validating runs/train/v11n_withback_12/weights/best.pt...
Ultralytics 8.3.56 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M2)
YOLO11n summary (fused): 238 layers, 2,583,322 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [01:34<00:00,  3.65s/it]


                   all        810       1744      0.625      0.622      0.624      0.496
                  dent        352        501      0.593      0.451       0.46       0.23
               scratch        431        728      0.489      0.475      0.459      0.253
                 crack        122        177      0.269      0.119      0.127     0.0603
         glass shatter        134        135      0.929       0.97      0.985      0.911
           lamp broken        139        141      0.636       0.83       0.82      0.671
             tire flat         59         62      0.834      0.889      0.897      0.851
Speed: 0.8ms preprocess, 114.7ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/train/v11n_withback_12


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x346bddc10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        